In [111]:
# part 1 (0) :
#==========================================================
# 📌 모듈 불러오기
#==========================================================
import os, sys, random, json, subprocess
from typing import List, Dict

import numpy as np

# LangChain
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document

from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.filterwarnings('ignore')
print("✅ 모듈 불러오기 - 완료")


✅ 모듈 불러오기 - 완료


In [112]:
# part 1 (1):
#==========================================================
# 📌 시드 고정(재현성)
#==========================================================
SEED = int(os.getenv("SEED", "2024"))
random.seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
try:
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
except Exception:
    pass
print("✅ 시드 고정 - 완료")


✅ 시드 고정 - 완료


In [113]:
# part 1 (2) 
#===============================================================================
# ▶ 작업환경 플래그
#===============================================================================
IS_GOOGLE = True if 'google.colab'           in sys.modules   else False
IS_KAGGLE = True if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ    else False
IS_LOCAL  = True if not (IS_GOOGLE or IS_KAGGLE)              else False
print(f"✅ IS_GOOGLE={IS_GOOGLE}, IS_KAGGLE={IS_KAGGLE}, IS_LOCAL={IS_LOCAL}")


✅ IS_GOOGLE=False, IS_KAGGLE=False, IS_LOCAL=True


In [114]:
# part 1 (3) 
#==========================================================
# 📌 상수설정 (모델/토큰/메시지) + api_token.txt 로드
#==========================================================
API_TOKEN_PATH  = os.getenv("API_TOKEN", "./api_token.txt")
OPENAI_API_KEY  = os.getenv("OPENAI_API_KEY", "")

if not OPENAI_API_KEY and os.path.exists(API_TOKEN_PATH):
    try:
        with open(API_TOKEN_PATH, "r", encoding="utf-8") as f:
            _k = f.read().strip()
        if _k:
            os.environ["OPENAI_API_KEY"] = _k
            OPENAI_API_KEY = _k
            print("✅ OpenAI API 키를 api_token.txt에서 로드했습니다.")
    except Exception as e:
        print(f"⚠️ API 토큰 파일 읽기 실패: {e}")

if OPENAI_API_KEY:
    print("✅ OpenAI API 키 준비됨")
else:
    print("⚠️ OPENAI_API_KEY 미설정 — 실행 전 반드시 설정 또는 api_token.txt 준비")

# 🔧 모델명
CHAT_MODEL_NAME  = os.getenv("OPENAI_CHAT_MODEL", "gpt-4o-mini")
EMBED_MODEL_NAME = os.getenv("OPENAI_EMBED_MODEL", "text-embedding-3-small")

# 🔢 토큰/파라미터
MAX_OUTPUT_TOKENS = int(os.getenv("MAX_OUTPUT_TOKENS", "800"))
TEMPERATURE       = float(os.getenv("TEMPERATURE", "0.2"))

# 🛡 도메인 제한 메시지
OFF_TOPIC_MESSAGE = "치매관련 상담만 가능합니다."

# 🧭 고정 질문 상수 (요청 형식)
Q1 = '자주 쓰던 물건 이름이 갑자기 생각안 난적이 있나요?'
Q2 = '대화중단어가 잘 떠오르지 않아서 곤란했던 적이 있나요?'
Q3 = '가족이나 지인이 평소와 다르다고 한적이 있나요?'
Q4 = '최근에 불편했던 점이나 걱정되는 점이 있나요?'

GUIDE_QUESTIONS = [Q1, Q2, Q3, Q4]
print("✅ 상수설정 - 완료")


✅ OpenAI API 키 준비됨
✅ 상수설정 - 완료


In [115]:
# part 2 (1) 
#==========================================================
# 📌 페르소나, 템플릿, 정책 (출력: <요약>만)
#==========================================================
from langchain_core.prompts import ChatPromptTemplate

SYSTEM_PERSONA = '''\
당신은 한국어로 상담하는 의학(치매진단) 상담 챗봇입니다.
- 역할: 환자/보호자의 서술(STT 텍스트)을 읽고, 지정된 템플릿으로 간결하고 체계적인 요약을 만듭니다.
- 태도: 정중하고 전문적이며, 과도한 확신/진단을 피하고 안전수칙을 강조합니다.
- 범위: 치매, 경도인지장애, 관련 증상/검사/일상 안전/가족 교육과 같은 주제에 한정합니다.
- 금지: 치매상담과 무관한 정보(예: 패스트푸드 신메뉴, 음료 당분 등)에 대한 응답 생성 금지.
- 출력: 반드시 아래 '요약 템플릿'의 <요약> 섹션만 출력합니다. (<질문>, <STT>는 출력하지 말 것)
'''

SUMMARY_TEMPLATE = '''\
<요약>
1. **주 증상**
{symptoms_bullets}

2. **상담내용**
{counselling_bullets}

3. **심리상태**
{psych_bullets}

4. **AI 해석**
{ai_interp_bullets}

5. **주의사항**
{caution_bullets}
'''

SUMMARISE_PROMPT = ChatPromptTemplate.from_messages(
    [
      ("system", SYSTEM_PERSONA + "\n\n"
        "아래 '참고 문서(요약)'는 최신 지식 보강을 위한 참고용입니다. "
        "근거가 불명확하면 과도한 단정 대신 조심스러운 해석을 제시하세요.\n"
        "반드시 '요약 템플릿' 구조를 그대로 유지하며 <요약> 섹션만 출력하십시오."),
      ("human",
       "가이드 질문(고정 4개 중 선택): {guide_question}\n\n"
       "참고 문서(요약):\n{rag_context}\n\n"
       "사용자 STT 원문:\n{transcript}\n\n"
       "요구사항:\n"
       "1) '주 증상'에는 핵심 증상만 '-' 불릿으로 간결히.\n"
       "2) '상담내용'에는 구체적 사건/진술 중심으로 불릿화.\n"
       "3) '심리상태'는 각 항목을 **2단계 불릿**으로 작성합니다:\n"
       "   - (상위 불릿) 감정 단어 1~3어 (예: 두려움, 불안, 수치심 등)\n"
       "     - 근거 : “STT에서 발췌한 6~20자 핵심 구절”\n"
       "   예) \n"
       "   - 두려움\n"
       "      - 근거 : “혼자 외출하는 것도 무섭습니다”\n"
       "4) 'AI 해석'은 병명 단정 금지(예: '~가능성 시사').\n"
       "5) '주의사항'은 일상 안전, 정서적 지지, 검진 권고 등을 포함.\n"
       "6) 반드시 아래 형식으로 출력(오직 <요약> 섹션만):\n"
       "{summary_template}\n"
       "출력 시 한국어 따옴표(“)를 유지하세요."
      )
    ]
)
print("✅ 페르소나/템플릿(요약 전용·심리상태 2단계) - 준비 완료")


✅ 페르소나/템플릿(요약 전용·심리상태 2단계) - 준비 완료


In [116]:
# part 2 (2) 
#==========================================================
# 📌 LLM/임베딩/웹검색(RAG) 구성 + Multi-Search + Rerank-Then-Summarise
#     - DuckDuckGo는 ddgs 직접 호출로 경고 제거
#==========================================================
import os, sys, subprocess, json
import numpy as np
from typing import List, Dict, Any
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.documents import Document

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")

llm = ChatOpenAI(
    model=CHAT_MODEL_NAME,
    temperature=TEMPERATURE,
    max_tokens=MAX_OUTPUT_TOKENS,
    api_key=OPENAI_API_KEY,
)
embeddings = OpenAIEmbeddings(model=EMBED_MODEL_NAME, api_key=OPENAI_API_KEY)

# --- 공용 유틸 ---
def _try_import(module_path: str, cls_name: str):
    try:
        mod = __import__(module_path, fromlist=[cls_name])
        return getattr(mod, cls_name)
    except Exception:
        return None

def _norm_list_result(engine: str, item: Dict[str, Any]) -> Document:
    title   = item.get("title") or item.get("name") or ""
    link    = item.get("link") or item.get("url") or item.get("href") or item.get("source") or ""
    snippet = item.get("snippet") or item.get("body") or item.get("content") or item.get("description") or ""
    text = f"{title}\n{snippet}\nURL: {link}"
    return Document(page_content=text, metadata={"source": link, "title": title, "engine": engine})

def _result_to_docs(engine_name: str, result: Any) -> List[Document]:
    docs: List[Document] = []
    if isinstance(result, list):
        for it in result:
            if isinstance(it, dict):
                docs.append(_norm_list_result(engine_name, it))
            else:
                docs.append(Document(page_content=str(it), metadata={"engine": engine_name}))
    else:
        docs.append(Document(page_content=str(result), metadata={"engine": engine_name}))
    return docs

# --- ddgs (DuckDuckGo native) ---
def _ensure_ddgs():
    try:
        from ddgs import DDGS
        return DDGS
    except Exception:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "ddgs"], stdout=subprocess.DEVNULL)
            from ddgs import DDGS
            return DDGS
        except Exception:
            return None

DDGS = _ensure_ddgs()

def ddgs_search(query: str, k: int = 5) -> List[Document]:
    if DDGS is None:
        return []
    docs: List[Document] = []
    try:
        with DDGS() as ddgs:
            # 참고: ddgs.text(...)는 dict 스트림을 반환 (title, href, body 등)
            for i, item in enumerate(ddgs.text(query, max_results=k)):
                title = item.get("title") or ""
                link  = item.get("href") or ""
                body  = item.get("body") or ""
                docs.append(Document(page_content=f"{title}\n{body}\nURL: {link}",
                                     metadata={"source": link, "title": title, "engine": "ddgs"}))
                if i+1 >= k:
                    break
    except Exception as e:
        print(f"⚠️ ddgs 검색 실패: {e}")
    return docs

# --- Google CSE / SerpAPI / Bing (있을 때만 사용) ---
GoogleSearchAPIWrapper = _try_import("langchain_community.utilities", "GoogleSearchAPIWrapper")
SerpAPIWrapper         = _try_import("langchain_community.utilities", "SerpAPIWrapper")
BingSearchAPIWrapper   = _try_import("langchain_community.utilities", "BingSearchAPIWrapper")
WikipediaLoader        = _try_import("langchain_community.document_loaders", "WikipediaLoader")
Chroma                 = _try_import("langchain_community.vectorstores", "Chroma")

def _safe_init_google():
    if GoogleSearchAPIWrapper and os.getenv("GOOGLE_API_KEY") and os.getenv("GOOGLE_CSE_ID"):
        try:
            return GoogleSearchAPIWrapper()
        except Exception:
            return None
    return None

def _safe_init_serpapi():
    if SerpAPIWrapper and os.getenv("SERPAPI_API_KEY"):
        try:
            return SerpAPIWrapper()
        except Exception:
            return None
    return None

def _safe_init_bing():
    if BingSearchAPIWrapper and os.getenv("BING_SUBSCRIPTION_KEY"):
        try:
            return BingSearchAPIWrapper()
        except Exception:
            return None
    return None

gse  = _safe_init_google()
serp = _safe_init_serpapi()
bing = _safe_init_bing()

def multi_engine_search(query: str, k: int = 5) -> List[Document]:
    docs: List[Document] = []
    # 우선순위: Google CSE → Bing → SerpAPI → ddgs
    if gse:
        try:
            res = gse.results(query, k)
            docs.extend(_result_to_docs("google", res))
        except Exception as e:
            print(f"⚠️ google 검색 실패: {e}")
    if bing:
        try:
            res = bing.results(query, k)
            docs.extend(_result_to_docs("bing", res))
        except Exception as e:
            print(f"⚠️ bing 검색 실패: {e}")
    if serp:
        try:
            res = serp.results(query, k)
            docs.extend(_result_to_docs("serpapi", res))
        except Exception as e:
            print(f"⚠️ serpapi 검색 실패: {e}")
    # ddgs는 항상 마지막 폴백으로 실행
    docs.extend(ddgs_search(query, k))

    # 간단 중복 제거
    seen = set()
    uniq_docs: List[Document] = []
    for d in docs:
        key = (d.metadata.get("source") or "") + "|" + (d.metadata.get("title") or "") + "|" + d.page_content[:80]
        if key in seen:
            continue
        seen.add(key)
        uniq_docs.append(d)
    return uniq_docs

def _embed_texts(texts: List[str]) -> List[List[float]]:
    return embeddings.embed_documents(texts)

def _embed_query(text: str) -> List[float]:
    return embeddings.embed_query(text)

def _cosine(a: List[float], b: List[float]) -> float:
    a = np.array(a, dtype=np.float32); b = np.array(b, dtype=np.float32)
    denom = (np.linalg.norm(a) * np.linalg.norm(b)) + 1e-12
    return float(np.dot(a, b) / denom)

def build_rag_context_rerank(
    transcript: str,
    queries: List[str],
    k_search: int = 8,
    k_vec: int    = 12,
    k_rerank: int = 8,
    k_final: int  = 4
) -> str:
    # 1) 멀티 엔진 검색 집계
    all_docs: List[Document] = []
    for q in queries:
        all_docs.extend(multi_engine_search(q, k=max(3, k_search // max(1, len(queries)) + 1)))

    # 2) 위키 보강
    if (WikipediaLoader is not None) and (len(all_docs) < 2):
        try:
            wiki_docs = WikipediaLoader(query="치매", load_max_docs=3).load()
            all_docs.extend(wiki_docs)
        except Exception as e:
            print(f"⚠️ Wikipedia 로드 실패: {e}")

    if not all_docs:
        return ""

    # 3) 1차 후보 (Chroma 사용 가능 시)
    if Chroma is not None:
        try:
            vs = Chroma.from_documents(all_docs, embeddings)
            retriever = vs.as_retriever(search_kwargs={"k": min(k_vec, max(1, len(all_docs)))})
            seed_query = " ".join(queries[:3]) or "치매 경도인지장애 초기 증상 안전 가족 교육 평가도구"
            vec_docs = retriever.invoke(seed_query)
        except Exception as e:
            print(f"⚠️ 벡터스토어/리트리버 생성 실패: {e}")
            vec_docs = all_docs[:k_vec]
    else:
        vec_docs = all_docs[:k_vec]

    # 4) 재랭크 (쿼리+원문 기반 코사인)
    combined_query = ((" ".join(queries)) + " " + transcript[:600]).strip()
    q_emb = _embed_query(combined_query)
    cand_docs  = vec_docs[:k_rerank]
    cand_texts = [d.page_content for d in cand_docs]
    cand_embs  = _embed_texts(cand_texts)
    scores = [(_cosine(q_emb, e), i) for i, e in enumerate(cand_embs)]
    scores.sort(key=lambda x: x[0], reverse=True)

    top_docs = [cand_docs[i] for (s, i) in scores[:k_final]]
    rag_text = "\n\n".join([d.page_content[:1200] for d in top_docs])
    return rag_text

print("✅ LLM/임베딩/멀티검색(+ddgs, 재랭크) - 준비 완료")


✅ LLM/임베딩/멀티검색(+ddgs, 재랭크) - 준비 완료


In [129]:
# part 2 (3) 
#==========================================================
# 📌 도메인 가드 (치매상담 범위 밖 → 강력 차단)
#  - 하드 블록 정규식 + 화이트리스트 패턴 + LLM 이진 분류(JSON)
#==========================================================
import re, json
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 1) 화이트리스트(치매/인지) 패턴 — 있으면 강력한 on-topic 신호
ALLOW_PATTERNS = [
    r"치매", r"경도인지장애", r"알츠하이머", r"인지(기능|저하)?",
    r"(기억|망각|건망)[가-힣\s]*문제", r"단기\s*기억", r"언어\s*유창성",
    r"단어가\s*잘\s*떠오르지", r"길(을)?\s*잃", r"방향감각", r"일상\s*생활(능력)?",
    r"MMSE", r"MoCA", r"신경\s*심리\s*검사", r"검사(를)?\s*받", r"보호자",
    r"(우울|불안|수치심|무기력)", r"혼자\s*외출(이)?\s*무섭",
]

# 2) 하드 블록 패턴 — 도메인 메타/능력/정체성 묻는 질문 등
HARD_BLOCK_PATTERNS = [
    r"너(는)?\s*(누구|뭐|뭔데)", r"당신(은)?\s*(누구|뭐|뭔데)",
    r"(상담|이거|이런(건)?)\s*(되|가능)(하|합)냐", r"상담\s*잘하냐",
    r"챗봇", r"\bAI\b", r"무엇이(냐|야)", r"뭐(야|냐)",
    r"테스트\s*(문|용)", r"아무말", r"장난", r"헛소리",
]

# 3) 소프트 블록(일반/일상) 힌트 — allow가 전혀 없고 길이 짧으면 차단 가중
SOFT_BLOCK_HINTS = [
    r"안녕|하이|헬로|ㅎㅇ", r"누구|정체|소개", r"도움|가능", r"어떻게|방법",
    r"상담\s*(가능|돼)|잘하", r"되긴\s*하냐", r"되나", r"되나요",
]

# 4) LLM 이진 분류 프롬프트 (엄격 JSON, 예시 포함)
CLASSIFY_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system",
         "너는 입력이 '치매/인지장애 상담' 범위에 해당하는지 이진 분류하는 필터다. "
         "반드시 JSON으로만 출력하라. 형식: {\"on_topic\": true|false, \"reason\": \"...\"} "
         "on_topic 기준: 치매/경도인지장애/기억/언어/방향감각/검사/일상 안전/가족 상담 등 **구체적 증상/걱정/행동**이 포함되어야 함. "
         "메타 질문(너는 누구냐, 상담 되냐 등)·일반 잡담·광고·일반 건강/식품 등은 false."
        ),
        ("human",
         "입력:\n\"\"\"\n{user_text}\n\"\"\"\n"
         "예시_off:\n- 너는 누구냐? -> false\n- 상담 잘하냐? -> false\n- 이런건 상담되긴 하냐? -> false\n"
         "예시_on:\n- 최근에 물건을 자주 잃어버립니다 -> true\n- 혼자 외출이 무서워 길을 잃을까 걱정돼요 -> true\n"
         "오직 JSON만 출력:")
    ]
)

classifier_llm = ChatOpenAI(
    model=CHAT_MODEL_NAME,
    temperature=0.0,
    max_tokens=80,
    api_key=os.getenv("OPENAI_API_KEY"),
)
classifier_chain = CLASSIFY_PROMPT | classifier_llm | StrOutputParser()

_allow_regex = [re.compile(p) for p in ALLOW_PATTERNS]
_hard_block_regex = [re.compile(p) for p in HARD_BLOCK_PATTERNS]
_soft_block_regex = [re.compile(p) for p in SOFT_BLOCK_HINTS]

def _has_any(regex_list, text: str) -> bool:
    return any(r.search(text) for r in regex_list)

def is_on_topic(text: str) -> bool:
    """강화된 온토픽 판별: 규칙→LLM 순서, 실패 시 보수적으로 차단."""
    if not text or not text.strip():
        return False

    t = text.strip()
    t_norm = re.sub(r"\s+", " ", t)

    # (A) 하드 블록: 하나라도 매칭되면 즉시 차단
    if _has_any(_hard_block_regex, t_norm):
        return False

    # (B) 화이트리스트 신호(의학/치매 관련) 체크
    allow_hits = sum(1 for r in _allow_regex if r.search(t_norm))

    # (C) 짧고(<= 25자) allow가 0이고, 소프트 블록 힌트가 있으면 차단
    if len(t_norm) <= 25 and allow_hits == 0 and _has_any(_soft_block_regex, t_norm):
        return False

    # (D) allow 신호가 충분(>=1)이면 우선 통과 시도하되, 너무 일반적 문장만 있으면 LLM 검증
    if allow_hits >= 1:
        # 간단한 일반/메타 질문만으로 허위 양성일 수 있으니 LLM 재확인
        try:
            raw = classifier_chain.invoke({"user_text": t_norm})
            data = json.loads(raw)
            return bool(data.get("on_topic", False))
        except Exception:
            # LLM 실패 시엔 보수적으로 True (실제 요약에서 의료 템플릿으로 수렴)
            return True

    # (E) 기본은 LLM 판별 (엄격 JSON, 실패 시 차단)
    try:
        raw = classifier_chain.invoke({"user_text": t_norm})
        data = json.loads(raw)
        return bool(data.get("on_topic", False))
    except Exception:
        return False

print("✅ 도메인 가드(강화판) - 준비 완료")


✅ 도메인 가드(강화판) - 준비 완료


In [130]:
# part 3 (1) 
#==========================================================
# 📌 검색질의 생성기 (STT → 관련 검색어 2~4개)
#==========================================================
QUERY_PROMPT = ChatPromptTemplate.from_messages(
    [
      ("system",
       "너는 의료 상담 보조 검색어 생성기이다. 입력 STT에서 핵심 증상/키워드를 뽑아 "
       "치매/경도인지장애와 관련된 한국어 검색질의 2~4개를 JSON 배열로만 출력하라."),
      ("human", "{transcript}")
    ]
)
query_llm   = ChatOpenAI(model=CHAT_MODEL_NAME, temperature=0.1, max_tokens=120, api_key=os.getenv("OPENAI_API_KEY"))
query_chain = QUERY_PROMPT | query_llm | StrOutputParser()

def make_search_queries(transcript: str) -> List[str]:
    try:
        raw = query_chain.invoke({"transcript": transcript})
        qs = json.loads(raw)
        qs = [q for q in qs if isinstance(q, str)]
        base = ["치매 초기 증상", "경도인지장애 언어 유창성", "일상 안전 가족 교육", "단기 기억력 저하 원인"]
        return (qs + base)[:4]
    except Exception:
        return ["치매 초기 증상", "경도인지장애 언어 유창성", "일상 안전 가족 교육", "단기 기억력 저하 원인"]

print("✅ 검색질의 생성기 - 준비 완료")


✅ 검색질의 생성기 - 준비 완료


In [131]:
# part 3 (2) 
#==========================================================
# 📌 요약 체인 (템플릿 강제, <요약>만) + 파일 저장 헬퍼
#==========================================================
summary_chain = SUMMARISE_PROMPT | llm | StrOutputParser()

def summarise_transcript_only_summary(
    transcript: str,
    guide_question_index: int = 3,  # 0~3 (Q1~Q4)
) -> str:
    if not is_on_topic(transcript):
        return OFF_TOPIC_MESSAGE

    guide_question = GUIDE_QUESTIONS[max(0, min(3, int(guide_question_index)))]
    # 1) 질의 생성
    queries = make_search_queries(transcript)
    # 2) Rerank-Then-Summarise RAG
    rag_context = build_rag_context_rerank(transcript, queries)

    # 3) 요약 생성 (오직 <요약> 섹션만)
    out = summary_chain.invoke({
        "transcript": transcript.strip(),
        "rag_context": rag_context.strip() if rag_context else "(문맥 없음)",
        "guide_question": guide_question,
        "summary_template": SUMMARY_TEMPLATE,
    })
    return out.strip()

def summarise_from_txt_and_save(input_path: str, output_path: str, guide_question_index: int = 3, encoding="utf-8") -> str:
    with open(input_path, "r", encoding=encoding) as f:
        text = f.read()
    summary_only = summarise_transcript_only_summary(text, guide_question_index=guide_question_index)
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(summary_only + "\n")
    return output_path

print("✅ 요약 체인 & 저장 함수 - 준비 완료")


✅ 요약 체인 & 저장 함수 - 준비 완료


In [132]:
# part 4 (demo)
#==========================================================
# 📌 데모 실행: './script_1.txt' → 'summary_1_.txt' 로 저장
#==========================================================
INPUT_TXT  = "./script_1.txt"
OUTPUT_TXT = "./summary_1_.txt"
QUESTION_INDEX = 3   # 0:Q1, 1:Q2, 2:Q3, 3:Q4  (요청대로 Q1~Q4 중 선택)

if not os.path.exists(INPUT_TXT):
    print(f"⚠️ {INPUT_TXT} 파일이 없습니다. 테스트용으로 데모 텍스트를 생성합니다.")
    demo_stt = (
        "요즘 물건을 어디에 두었는지 자꾸 잊어버려서 힘듭니다. "
        "안경이나 휴대폰을 두고도 찾지를 못해서 가족들한테 자주 물어보게 돼요. "
        "또 밖에 나갔다가 길을 잃을까 봐 혼자 외출하는 것도 무섭습니다. "
        "집에서는 전화 통화를 하고 나면 금방 무슨 이야기를 했는지 기억이 안 나서 답답하고요. "
        "제 상태가 점점 더 나빠져서 가족들에게 짐이 될까 봐 가장 걱정입니다. "
        "대화할 때 단어가 잘 떠오르지 않아서 사람들이 이상하게 생각할까 봐 창피하기도 합니다."
    )
    with open(INPUT_TXT, "w", encoding="utf-8") as f:
        f.write(demo_stt)

out_path = summarise_from_txt_and_save(INPUT_TXT, OUTPUT_TXT, guide_question_index=QUESTION_INDEX)
print(f"✅ 저장 완료: {out_path}")
with open(out_path, 'r', encoding='utf-8') as f:
    preview = f.read()
print('------ 미리보기 ------')
print(preview[:1200])


✅ 저장 완료: ./summary_1_.txt
------ 미리보기 ------
<요약>
1. **주 증상**
- 물건을 잃어버림
- 길을 잃을까 두려움
- 전화 통화 후 기억 상실
- 단어가 잘 떠오르지 않음

2. **상담내용**
- 물건을 두고 찾지 못해 가족에게 자주 물어봄
- 혼자 외출하는 것이 두려움
- 전화 통화 후 대화 내용을 기억하지 못함
- 단어가 잘 떠오르지 않아 창피함

3. **심리상태**
- 두려움
  - 근거 : “혼자 외출하는 것도 무섭습니다”
- 불안
  - 근거 : “제 상태가 점점 더 나빠져서 가족들에게 짐이 될까 봐 가장 걱정입니다”
- 수치심
  - 근거 : “사람들이 이상하게 생각할까 봐 창피하기도 합니다”

4. **AI 해석**
- 현재 겪고 계신 증상은 경도인지장애(MCI)와 관련이 있을 수 있으며, 조기 검진이 필요할 수 있습니다.

5. **주의사항**
- 일상에서 물건을 정해진 장소에 두는 습관을 기르세요.
- 외출 시 안전한 경로를 미리 계획하고, 필요 시 동반자를 요청하세요.
- 정서적 지지를 위해 가족과의 소통을 강화하고, 전문가와 상담하여 검진을 받는 것이 좋습니다.



In [133]:
# part 5 (optional cli)
#==========================================================
# 📌 간단 대화 루프 (오프토픽 차단 + 요약)
#==========================================================
# 사용법:
#  - 첫 발화는 4개 고정 질문 중 하나를 출력
#  - 이후 사용자의 긴 응답(텍스트)을 받아 요약 템플릿으로 정리

def start_chat(guide_question_index: int = 3):
    q = GUIDE_QUESTIONS[max(0, min(3, int(guide_question_index)))]
    print(f"[상담 챗봇] {q}")
    print("[안내] 긴 답변을 입력하세요. '/quit' 입력 시 종료.")
    while True:
        user = input("\n[사용자] ").strip()
        if user.lower() in {"/quit", "quit", "exit"}:
            print("종료합니다.")
            break
        result = summarise_transcript(user, guide_question_index=guide_question_index)
        print("\n[요약]\n")
        print(result)

# start_chat(guide_question_index=3)  # ← 필요 시 주석 해제하여 실행
print("✅ (선택) CLI 루프 준비됨 — 필요 시 start_chat() 실행")


✅ (선택) CLI 루프 준비됨 — 필요 시 start_chat() 실행


In [142]:
# part 6
#==========================================================
# 📌 간단 대화 루프 (오프토픽 차단 + 요약)
#==========================================================
print(summarise_from_txt("./script_2.txt", guide_question_index=3))


치매관련 상담만 가능합니다.
